In [1]:
import numpy as np
import cvxpy as cp
import pickle

from problems.data_generation_SC import generate_problem_data
from problems.problem_definition_SC import BilevelProblem
from algorithms.barrier_blo import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA
from algorithms.BSG import BSG

n = 60
seed = 9

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)

max_elapsed_time = 120

x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
hparams = {
        'barrier_blo': {
            'M': 0.001,
            't': 0.01,
            'alpha_x': 0.0015,
            'alpha_y': 0.1,
            'beta_y': 1,
            'epsilon_x': 0.1,
            'epsilon_y': 0.05,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        },
        'blocc': {
            'gamma': 10,
            'alpha_x': 0.0001,
            'alpha_g_y': 0.0001,
            'alpha_F_y': 0.0001,
            'beta_g_y': 3,
            'beta_F_y': 3,
            'epsilon_x': 0.1,
            'epsilon_inner_y_g': 0.01,
            'epsilon_outer_y_g': 0.01,
            'epsilon_inner_y_F': 0.01,
            'epsilon_outer_y_F': 0.01,
            'maxmin_g_outer_max_iters': 5000000,
            'maxmin_F_outer_max_iters': 5000000,
            'maxmin_g_inner_max_iters': 5000000,
            'maxmin_F_inner_max_iters': 5000000,
            'main_max_iters': 1000
        },
        'bic_gaffa':{
            'alpha':0.000001,
            'c':1,
            'tau':1.1,
            'gamma_1':10,
            'gamma_2':1,
            'eta':1,
            'r':1,
            'epsilon':0.1,
            'max_iters':300
        },
        'BSG':{
            'alpha_x':0.00005,
            'alpha_y':0.00002,
            'alpha_z':3,
            'epsilon_x':0.1,
            'epsilon_y':0.01,
            'epsilon_z':0.01,
            'max_iters_x':1000000,
            'max_iters_y':1000000,
            'max_iters_z':1000000
        },
        'IGD': {
            'M': 1e-3,
            't': 1e-3,
            'alpha_x': 0.001,
            'alpha_y': 0.001,
            'epsilon_x': 1e-3,
            'epsilon_y': 1e-3,
            'inner_max_iters': 1000,
            'outer_max_iters': 1000
        }
    }

(CVXPY) Dec 16 05:18:16 PM: Encountered unexpected exception importing solver DIFFCP:
ImportError('diffcp >= 1.0.15 is required')


# BSG Algorithm

In [2]:
bsg_algo = BSG(problem, hparams)
x_opt_bsg, y_opt_bsg, history_bsg = bsg_algo.bsg(x_init, y_init, z_init, max_elapsed_time)
f_bsg_num = problem.f(x_opt_bsg, y_opt_bsg)
print(f_bsg_num)

f(x, y) = 100.61868913832755, grad_norm of hyperfunction= 76.59684148513914
f(x, y) = 100.34576002213024, grad_norm of hyperfunction= 76.13071594616541
f(x, y) = 100.06573879237398, grad_norm of hyperfunction= 75.66513630616419
f(x, y) = 99.7852657765065, grad_norm of hyperfunction= 75.20225243913082
f(x, y) = 99.50716137728708, grad_norm of hyperfunction= 74.74219281253926
f(x, y) = 99.23216297951036, grad_norm of hyperfunction= 74.28495833843448
f(x, y) = 98.96044064678597, grad_norm of hyperfunction= 73.83053649663778
f(x, y) = 98.69200982457718, grad_norm of hyperfunction= 73.37890149157523
f(x, y) = 98.42684770813511, grad_norm of hyperfunction= 72.93004694480688
f(x, y) = 98.16491713936358, grad_norm of hyperfunction= 72.48394549227022
f(x, y) = 97.906181439587, grad_norm of hyperfunction= 72.04059048371099
f(x, y) = 97.65060080586409, grad_norm of hyperfunction= 71.59995791957311
f(x, y) = 97.398137031198, grad_norm of hyperfunction= 71.16203240237022
f(x, y) = 97.1487520991508,

KeyboardInterrupt: 

# BSG Feasibility Check

In [119]:
barrier_algo = BarrierBLO(problem, hparams)
y_original_opt_bsg = barrier_algo.Interior_inner_loop(x_opt_bsg, y_opt_bsg)
f_bsg_theo = problem.f(x_opt_bsg, y_original_opt_bsg)
g_bsg_theo = problem.g(x_opt_bsg, y_original_opt_bsg)
g_bsg_num = problem.g(x_opt_bsg, y_opt_bsg)

h1_values = [problem.h_1(x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h2)]
h_total_bsg = sum(max(0, val) for val in h1_values + h2_values)

print(f"f_theoretical_opt_result={f_bsg_theo}, f_bsg_numerical_result={f_bsg_num}")
print(f"g_theoretical_opt_result={g_bsg_theo}, g_bsg_numerical_result={g_bsg_num}")
print(h_total_bsg)

f_theoretical_opt_result=-0.0011331233206135494, f_bsg_numerical_result=0.11685343661460479
g_theoretical_opt_result=-19.096307090180943, g_bsg_numerical_result=-19.331593512082257
0.004310501234829615


# BiC_GAFFA Algorithm

In [120]:
# bic_algo = BiC_GAFFA(problem, hparams)
# x_opt_bic, y_opt_bic, history_bic = bic_algo.bic_gaffa(x_init, y_init, z_init, theta_init, max_elapsed_time)
# f_bic_num = problem.f(x_opt_bic, y_opt_bic)
# print(f_bic_num)

# BiC_GAFFA Feasibility Check

In [121]:
# barrier_algo = BarrierBLO(problem, hparams)
# y_original_opt_bic = barrier_algo.Interior_inner_loop(x_opt_bic, y_opt_bic)
# f_bic_theo = problem.f(x_opt_bic, y_original_opt_bic)
# g_bic_theo = problem.g(x_opt_bic, y_original_opt_bic)
# g_bic_num = problem.g(x_opt_bic, y_opt_bic)

# h1_values = [problem.h_1(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h1)]
# h2_values = [problem.h_2(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h2)]
# h_total_bic = sum(max(0, val) for val in h1_values + h2_values)

# print(f"f_theoretical_opt_result={f_bic_theo}, f_bic_numerical_result={f_bic_num}")
# print(f"g_theoretical_opt_result={g_bic_theo}, g_bic_numerical_result={g_bic_num}")
# print(h_total_bic)

# BLOCC Algorithm

In [122]:
blocc_algo = BLOCC(problem, hparams)
x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time)
f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
print(f_blocc_num)

Main iteration 1
f(x,y)=62.82273001094169,grad_norm = 76.46518414045083
Main iteration 2
f(x,y)=62.25570893150242,grad_norm = 75.53167942345445
Main iteration 3
f(x,y)=61.70241959263856,grad_norm = 74.61054392614139
Main iteration 4
f(x,y)=61.16252596254482,grad_norm = 73.70069123897372
Main iteration 5
f(x,y)=60.635702838061704,grad_norm = 72.80198255158362
Main iteration 6
f(x,y)=60.12163293028699,grad_norm = 71.91428077088014
Main iteration 7
f(x,y)=59.62000667152323,grad_norm = 71.03745049765327
Main iteration 8
f(x,y)=59.13052202694617,grad_norm = 70.17135800555512
Main iteration 9
f(x,y)=58.479885415465056,grad_norm = 168.4272745798325
Main iteration 10
f(x,y)=47.84188662133841,grad_norm = 1298.0851630583782
Main iteration 11
f(x,y)=17.75475591470757,grad_norm = 471.86501732875064
Main iteration 12
f(x,y)=4.33138503538985,grad_norm = 306.15319048946526
Main iteration 13
f(x,y)=-1.2082412004689616,grad_norm = 196.03874829699234
Main iteration 14
f(x,y)=-3.6108151311211216,grad_nor

# BLOCC Feasibility Check

In [123]:
barrier_algo = BarrierBLO(problem, hparams)
y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
f_blocc_theo = problem.f(x_opt_blocc, y_original_opt_blocc)
g_blocc_theo = problem.g(x_opt_blocc, y_original_opt_blocc)
g_blocc_num = problem.g(x_opt_blocc, y_opt_blocc)

h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]
h_total_blocc = sum(max(0, val) for val in h1_values + h2_values)

print(f"f_theoretical_opt_result={f_blocc_theo}, f_blocc_numerical_result={f_blocc_num}")
print(f"g_theoretical_opt_result={g_blocc_theo}, g_blocc_numerical_result={g_blocc_num}")
print(h_total_blocc)


f_theoretical_opt_result=-4.705512582367909, f_blocc_numerical_result=-9.845718815714047
g_theoretical_opt_result=-16.1418999136807, g_blocc_numerical_result=-16.830739834842717
0.12170400762696088


# BFBM Algorithm

In [124]:
barrier_algo = BarrierBLO(problem, hparams)
x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(x_init, y_init, max_elapsed_time)
f_barrier_num = problem.f(x_opt_barrier, y_opt_barrier)
print(f_barrier_num)


Outer iteration 1
Inner loop converged at iteration 70
f(x, y) = 92.70382549956263, grad_norm of hyperfunction= 77.86176379872232
Outer iteration 2
Inner loop converged at iteration 48
f(x, y) = 107.6915910874211, grad_norm of hyperfunction= 683.7036958592544
Outer iteration 3
Inner loop converged at iteration 81
f(x, y) = 42.80036026352853, grad_norm of hyperfunction= 157.62495482297018
Outer iteration 4
Inner loop converged at iteration 77
f(x, y) = 25.426597816265208, grad_norm of hyperfunction= 139.25909029460863
Outer iteration 5
Inner loop converged at iteration 76
f(x, y) = 13.932894704449865, grad_norm of hyperfunction= 116.17348667648194
Outer iteration 6
Inner loop converged at iteration 75
f(x, y) = 6.703014627407612, grad_norm of hyperfunction= 98.4301597175037
Outer iteration 7
Inner loop converged at iteration 73
f(x, y) = 1.9146128591150529, grad_norm of hyperfunction= 84.09538336937626
Outer iteration 8
Inner loop converged at iteration 73
f(x, y) = -1.3410736777439425,

# BFBM Feasibility Check

In [125]:
y_original_opt_barrier = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
f_barrier_theo = problem.f(x_opt_barrier, y_original_opt_barrier)
g_barrier_theo = problem.g(x_opt_barrier, y_original_opt_barrier)
g_barrier_num = problem.g(x_opt_barrier, y_opt_barrier)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
print(h_total_barrier)

f_theoretical_opt_result=-11.737845758473632, f_barrier_numerical_result=-11.773866513027386
g_theoretical_opt_result=-23.77375552372185, g_barrier_numerical_result=-23.67386056790901
0


# Save Data

In [126]:
results = {
    'seed': seed,
    'bsg': {
        'x_opt': x_opt_bsg,
        'y_opt': y_opt_bsg,
        'history': history_bsg,
        'y_original_opt': y_original_opt_bsg,
        'f_theoretical_opt': f_bsg_theo,
        'f_numerical': f_bsg_num,
        'g_theoretical_opt': g_bsg_theo,
        'g_numerical': g_bsg_num,
        'h_total': h_total_bsg
    },
    'blocc': {
        'x_opt': x_opt_blocc,
        'y_opt': y_opt_blocc,
        'history': history_blocc,
        'y_original_opt': y_original_opt_blocc,
        'f_theoretical_opt': f_blocc_theo,
        'f_numerical': f_blocc_num,
        'g_theoretical_opt': g_blocc_theo,
        'g_numerical': g_blocc_num,
        'h_total': h_total_blocc
    },
    'barrier': {
        'x_opt': x_opt_barrier,
        'y_opt': y_opt_barrier,
        'history': history_barrier,
        'y_original_opt': y_original_opt_barrier,
        'f_theoretical_opt': f_barrier_theo,
        'f_numerical': f_barrier_num,
        'g_theoretical_opt': g_barrier_theo,
        'g_numerical': g_barrier_num,
        'h_total': h_total_barrier
    }
}

save_filename = f'results_SC_seed_{seed}.pkl'

with open(save_filename, 'wb') as f:
    pickle.dump(results, f)

print(f"Results saved to {save_filename}")

Results saved to results_SC_seed_9.pkl


In [128]:
algos = ['bsg', 'blocc', 'barrier']

for seed in range(10):
    filename = f'results_SC_seed_{seed}.pkl'
    with open(filename, 'rb') as f:
        results = pickle.load(f)

    for algo in algos:
        f_num = results[algo]['f_numerical']
        f_theo = results[algo]['f_theoretical_opt']
        g_num = results[algo]['g_numerical']
        g_theo = results[algo]['g_theoretical_opt']
        h_total = results[algo]['h_total']
        
        g_diff = g_num - g_theo
        
        print(f"Seed={seed}, Algo={algo}, f_num={f_num}, f_theo={f_theo}, g_num-g_theo={g_diff}, h_total={h_total}")

Seed=0, Algo=bsg, f_num=26.0437380951962, f_theo=25.741990419664276, g_num-g_theo=-0.3735792311873922, h_total=0.007598663373320151
Seed=0, Algo=blocc, f_num=-5.69181108554055, f_theo=-2.0428905989612183, g_num-g_theo=0.49246340326545557, h_total=0.04706209744849421
Seed=0, Algo=barrier, f_num=-8.095506392335196, f_theo=-8.137383788803952, g_num-g_theo=0.05489628894256704, h_total=0
Seed=1, Algo=bsg, f_num=-7.7070585964560685, f_theo=-7.687151614951677, g_num-g_theo=0.021112079743772938, h_total=0.0007968080472182636
Seed=1, Algo=blocc, f_num=-13.644086519251438, f_theo=-8.944923305818726, g_num-g_theo=0.13690076916577887, h_total=0.08946861119989946
Seed=1, Algo=barrier, f_num=-15.127794402431228, f_theo=-15.205895981994757, g_num-g_theo=0.056883880895874483, h_total=0
Seed=2, Algo=bsg, f_num=16.550802934381856, f_theo=16.488519343121023, g_num-g_theo=-0.10843519198870055, h_total=0.0019313619126460624
Seed=2, Algo=blocc, f_num=-16.5787485939732, f_theo=-12.904871683147174, g_num-g_th